# Regression for the Petite Couronne region

Correlation doesnt require any sort of regression. Then we just want to find a fine predictor.

We investigate the correlation between the aggregated 2SFCA score and mean income in the Petite Couronne region

In [ ]:
gdf_name = "results_pcparis.gpkg" 

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.environ['USE_PYGEOS'] = '0'
import pysal.lib
import helpers as hs
from importlib import reload
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%cd ../data
gdf = gpd.read_file(gdf_name, layer="cool")

## Regression with single variable

In [ ]:
gdf["mean_income"] = gdf["Ind_snv"]/gdf["Ind"]

In [ ]:
gdf.plot(x='mean_income',y='CS_aggregated', kind='scatter')
plt.savefig("foo.png")
plt.show()

We see that there is a disproportionate amount of extremely low accessibility scores. Therefore, we transform the y axis to logscale:

In [ ]:
gdf.plot(x='mean_income',y='CS_aggregated', kind='scatter')
plt.yscale("log")
plt.show()

Now we begin to see some trend lines. On this log transformed data, we can perform linear.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

x = gdf["mean_income"].values
y = gdf["CS_aggregated"].values
x = x.reshape(-1,1)
yr = y.reshape(-1,1)

yr = yr + 0.0001
yr = np.log(yr)

model = LinearRegression()
model.fit(x, yr)

yr_pred = model.predict(x)

plt.scatter(x, yr, color='blue', label='Actual Data')
plt.plot(x, yr_pred, color='red', label='Fitted Line')
plt.xlabel('Log of Mean Income')
plt.ylabel('Log of CS Aggregated')
plt.legend()
plt.title('Log-Transformed Linear Regression')
plt.show()

There is a large amount of outliers still with extremely low CS aggregated score. This may result from a large amount of squares being on the edge of the considered area. As the bounds of the surveyed area are quite rough, those outliers might happen and it makes sense to remove them.

In [ ]:
threshold = gdf['CS_aggregated'].quantile(0.01)
outliers = gdf[gdf['CS_aggregated'] < threshold]
remaining = gdf[gdf['CS_aggregated'] >= threshold]

income_threshold = remaining['mean_income'].quantile(0.999)
rem2 = remaining[remaining['mean_income'] < income_threshold]

In [ ]:
outliers.explore()

In [ ]:

x = rem2["mean_income"].values
y = rem2["CS_aggregated"].values
x = x.reshape(-1,1)
yr = y.reshape(-1,1)

yr = yr + 0.0001
yr = np.log(yr)

model = LinearRegression()
model.fit(x, yr)

yr_pred = model.predict(x)

print(mean_squared_error(yr, yr_pred))

plt.scatter(x, yr, color='blue', label='Actual Data')
plt.plot(x, yr_pred, color='red', label='Fitted Line')
plt.xlabel('Mean Income')
plt.ylabel('Log of CS Aggregated')
plt.legend()
plt.title('Log-Transformed Linear Regression')
plt.savefig("foo2.png")
plt.show()

Clear tend, but with a huge variance.

Why *not* remove the outleirs? - rich people often choose to live in inaccessible places.

## Regression with Mutliple Variables

In [ ]:
import seaborn as sns

In [ ]:
# Plot distribution
sns.histplot(gdf['CS_aggregated'], kde=True, bins=10)  # kde=True adds a kernel density estimate curve
plt.title('Distribution of Values')
plt.xlabel('values')
plt.ylabel('Frequency')
plt.show()

In [ ]:
gdf["CSlog"] = np.log(gdf["CS_aggregated"] + 0.00001)

In [ ]:
# Plot distribution
sns.histplot(gdf['CSlog'], kde=True, bins=20)  # kde=True adds a kernel density estimate curve
plt.title('Distribution of Values')
plt.xlabel('values')
plt.ylabel('Frequency')
plt.show()

we can see a ton of outliers!

In [ ]:
rem2["CSlog"] = np.log(rem2["CS_aggregated"] + 0.00001)

In [ ]:
# Plot distribution
sns.histplot(rem2['CSlog'], kde=True, bins=20)  # kde=True adds a kernel density estimate curve
plt.title('Distribution of Values')
plt.xlabel('values')
plt.ylabel('Frequency')
plt.show()

Looking good, almost normal!

In [ ]:
gdfs = rem2.copy()
gdfs["%_soc.minimum"] = (gdfs["Men_pauv"]/gdfs["Ind"])*100
gdfs["%_>_65"] = ((gdfs["Ind_65_79"] +gdfs["Ind_80p"])/gdfs["Ind"])*100
gdfs["%_<_17"] = ((gdfs["Ind_0_3"] +gdfs["Ind_4_5"] + gdfs["Ind_6_10"] + gdfs["Ind_11_17"])/gdfs["Ind"])*100
gdfs["%_<_bat_45"] = (gdfs["Log_av45"]/(gdfs["Log_av45"] +gdfs["Log_45_70"] + gdfs["Log_70_90"] + gdfs["Log_ap90"] + gdfs["Log_inc"]))*100
gdfs["%_>_bat_90"] = (gdfs["Log_ap90"]/(gdfs["Log_av45"] +gdfs["Log_45_70"] + gdfs["Log_70_90"] + gdfs["Log_ap90"] + gdfs["Log_inc"]))*100
gdfs["%_residences"] = (gdfs["Men_coll"]/(gdfs["Men_coll"] +gdfs["Men_mais"]))*100
gdfs["mean_income"] = gdfs["Ind_snv"]/gdfs["Ind"]
gdfs["density"] = gdfs["Ind"]/(0.0002)

In [ ]:
gdfs.describe()

In [ ]:
soc_vars = ["%_soc.minimum", "%_>_65", "%_<_17", "%_<_bat_45", "%_>_bat_90", "%_residences", "mean_income","density"]
for feature in soc_vars:
    plt.figure(figsize=(4, 3))
    plt.scatter(gdfs[feature].values, gdfs["CSlog"])
    plt.xlabel(feature)
    plt.tight_layout()

Okay. We only see some trends on: density, mean income, residences, bat90, bat45, social minimum. Let's see how correlated they are.

In [ ]:
soc_vars = ["%_soc.minimum", "%_>_65", "%_<_17", "%_<_bat_45", "%_>_bat_90", "%_residences", "mean_income","density"]
for feature in soc_vars:
    sns.histplot(gdfs[feature], kde=True, bins=20)  # kde=True adds a kernel density estimate curve
    plt.title('Distribution of Values')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
gdfs[soc_vars].describe()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
soc_vars = ["%_soc.minimum","%_<_bat_45", "%_>_bat_90", "%_residences", "mean_income","density"]

gdfrel = gdfs[soc_vars]
correlation_matrix = gdfrel.corr()
print("Correlation Matrix:")
print(correlation_matrix)

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

X = gdfrel  # Independent variables
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("VIF Scores:")
print(vif_data)

Generally, we get a semi-worrying correlation between density and number of residences. This someone violates the assumptions of linear regressions, but since it is one mutlicolinearity, we shall allow it.

In [ ]:
socvars = ["density", "%_>_bat_90", "%_<_bat_45", "%_residences", "mean_income"]
import statsmodels.api as sm

X = gdfs[socvars]  
X = sm.add_constant(X)  
y = gdfs['CSlog']  

model = sm.OLS(y, X).fit()

print(model.summary())

# Spatial regression recreated

In [ ]:
from pysal.model import spreg
from pysal.lib import weights
from spreg import OLS

In [ ]:
gdfs = gdf.copy()

In [ ]:
gdfs["%_soc.minimum"] = (gdfs["Men_pauv"]/gdfs["Ind"])*100
gdfs["%_>_65"] = ((gdfs["Ind_65_79"] +gdfs["Ind_80p"])/gdfs["Ind"])*100
gdfs["%_<_17"] = ((gdfs["Ind_0_3"] +gdfs["Ind_4_5"] + gdfs["Ind_6_10"] + gdfs["Ind_11_17"])/gdfs["Ind"])*100
gdfs["%_<_bat_45"] = (gdfs["Log_av45"]/(gdfs["Log_av45"] +gdfs["Log_45_70"] + gdfs["Log_70_90"] + gdfs["Log_ap90"] + gdfs["Log_inc"]))*100
gdfs["%_>_bat_90"] = (gdfs["Log_ap90"]/(gdfs["Log_av45"] +gdfs["Log_45_70"] + gdfs["Log_70_90"] + gdfs["Log_ap90"] + gdfs["Log_inc"]))*100
gdfs["%_residences"] = (gdfs["Men_coll"]/(gdfs["Men_coll"] +gdfs["Men_mais"]))*100
gdfs["mean_income"] = gdfs["Ind_snv"]/gdfs["Ind"]
gdfs["density"] = gdfs["Ind"]/(0.0002)

In [ ]:
w_queen = weights.contiguity.Queen.from_dataframe(gdfs) # adjacent in all directions, including diagonal.
# Replace Queen by rook for only up/down/left/right adjancy

In [ ]:
w_queen.transform = 'r'

Model B:

In [ ]:
dep_var = "CS_aggregated"
independ_var_B = ["%_soc.minimum", "%_>_65", "%_<_17", "%_<_bat_45", "%_>_bat_90", "%_residences", "mean_income","density"]
ols = OLS(gdfs[[dep_var]].values, gdfs[independ_var_B].values)

In [ ]:
sample = gdfs[independ_var_B]
sample.describe()

The density and mean income being distributed differently pose an issue. Therefore, we redistribute them (just for interpretability).

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
changevars = independ_var_B
gdfs[changevars] = scaler.fit_transform(gdfs[changevars])
gdfs["CS_aggregated"] += 0.0000001

In [ ]:
gdfs["CS_aggregated"] = scaler.fit_transform(gdfs[["CS_aggregated"]])

In [ ]:
mB = spreg.GM_Combo(
    gdfs[[dep_var]].values,
    gdfs[independ_var_B].values,
    w=w_queen,
    name_y=dep_var,
    name_x=independ_var_B,
)

print(mB.summary)

In [ ]:
mB = spreg.GM_Combo(
    gdfs[[dep_var]].values,
    gdfs[independ_var_B].values,
    w=w_queen,
    name_y=dep_var,
    name_x=independ_var_B,
)

print(mB.summary)

In [ ]:
m1 = spreg.OLS(
    gdfs[[dep_var]].values,
    gdfs[independ_var_B].values,
    name_y=dep_var,
    name_x=independ_var_B,
)

print(m1.summary)

In [ ]:
allvars = independ_var_B + ["CS_aggregated"]
allvar = gdfs[allvars]
allvar.describe()